In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Download latest version
path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

print("Path to dataset files:", path)

# Load data
df_missiles_daily = pd.read_csv(f"{path}/missile_attacks_daily.csv")
df_missiles_description = pd.read_csv(f"{path}/missiles_and_uav.csv")



/home/enzo/repos/projeto-final-data-science-2025-01/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/enzo/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136


In [4]:
# Salvar uma cópia do dataset na raiz do projeto
df_missiles_daily.to_excel("./data_outputs/missile_attacks_daily.xlsx", index=False)
df_missiles_description.to_excel("./data_outputs/missiles_and_uav.xlsx", index=False)

In [5]:
df_missiles = df_missiles_daily.merge(df_missiles_description, how='left', on=['model'])
df_missiles['date_start'] = pd.to_datetime(df_missiles['time_start'], format='mixed').dt.date.astype('datetime64[ns]')
print(df_missiles.shape)
display(df_missiles)
df_missiles.to_excel("./data_outputs/df_missiles.xlsx",index=False)

(2581, 31)


,time_start,time_end,model,launch_place,target,carrier,launched,destroyed,not_reach_goal,cross_border_belarus,...,type,launch_platform,name,name_NATO,in_sevice,designer,manufacturer,guidance_system,unit_cost,date_start
0,2025-06-21 21:00,2025-06-22 09:00,Shahed-136/131,Primorsko-Akhtarsk and Kursk oblast,Ukraine,NaN,47.0,18.0,10.0,NaN,...,loitering munition,NaN,NaN,NaN,NaN,NaN,Shahed Aviation Industries,NaN,NaN,2025-06-21
1,2025-06-21 21:00,2025-06-22 09:00,Iskander-M/KN-23,Voronezh oblast and Rostov oblast,Ukraine,NaN,2.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-06-21
2,2025-06-21 21:00,2025-06-22 09:00,C-300,Donetsk oblast,Ukraine,NaN,1.0,0.0,NaN,NaN,...,NaN,NaN,S-300,SA-10 Grumble,NaN,NaN,NaN,NaN,NaN,2025-06-21
3,2025-06-20 20:00,2025-06-21 10:00,Shahed-136/131,"Primorsko-Akhtarsk and Chauda, Crimea and Kurs...",Ukraine,NaN,272.0,140.0,112.0,NaN,...,loitering munition,NaN,NaN,NaN,NaN,NaN,Shahed Aviation Industries,NaN,NaN,2025-06-20
4,2025-06-21 02:40,2025-06-21 04:10,Kalibr,Black Sea,Poltava oblast,Navi,4.0,3.0,1.0,NaN,...,NaN,"naval ships, submarines, containers, airplanes...",Kalibr,SS-N-27 Sizzler,NaN,NaN,NaN,NaN,NaN,2025-06-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,2022-10-01 12:00,2022-10-01 12:00,Orlan-10,NaN,Ukraine,NaN,1.0,1.0,NaN,NaN,...,reconnaissance,NaN,NaN,NaN,NaN,NaN,Special Technology Center,NaN,NaN,2022-10-01
2577,2022-09-30,2022-09-30,Orlan-10,NaN,Ukraine,NaN,3.0,3.0,NaN,NaN,...,reconnaissance,NaN,NaN,NaN,NaN,NaN,Special Technology Center,NaN,NaN,2022-09-30
2578,2022-09-29 23:00,2022-09-29 23:59,Shahed-136/131,NaN,Odesa oblast and Mykolaiv oblast,NaN,7.0,5.0,NaN,NaN,...,loitering munition,NaN,NaN,NaN,NaN,NaN,Shahed Aviation Industries,NaN,NaN,2022-09-29
2579,2022-09-28,2022-09-28,X-59,NaN,Dnipropetrovsk oblast and Zaporizhzhia oblast,Su-35,5.0,4.0,NaN,NaN,...,"air-launched, air-to-surface, anti-ship, land-...","Su-30MK, Su-24M, MiG-27, Su-17M3/22M4, HAL Tej...",Kh-59 Ovod,AS-13 'Kingbolt',1980.0,MKB Raduga,Tactical Missiles Corporation,NaN,NaN,2022-09-28


In [ ]:
df_ = df_missiles.groupby(['category', pd.Grouper(key='date_start', axis=0, freq='M')], dropna=False)\
                 .agg({'launched': sum, 'destroyed': sum, 'model': set})
df_['destroyed_rario'] = round(df_['destroyed']/df_['launched']*100, 2)
df_ = df_.reset_index()
df_
for category in df_[df_['category'].notna()]['category'].unique():
    print(category)
    display(df_[(df_['category']==category)].set_index('date_start'))
    
display(df_[(df_['category'].isna())].set_index('date_start'))

In [ ]:
df__ = df_missiles.pivot_table(index='date_start', columns='category', values='launched', aggfunc='sum')
df__['Total'] = df__.sum(axis=1)
df__.fillna('').sort_values('Total', ascending=False)[:20]